In [4]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os # accessing directory structure


In [7]:
DATASET_PATH = "G:/python_workspaces/fashionRecommendation/Fashion-Recommendation/myntradataset/"
print(os.listdir(DATASET_PATH))

['images', 'styles.csv']
